In [1]:
import tensorflow as tf
import os
import cv2 #pip3 install opencv-python

from tensorflow import keras
import random
from keras.preprocessing.image import ImageDataGenerator

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from PIL import Image, ImageOps, ImageFilter, ImageEnhance  #PILLOW
from math import floor, ceil, pi
import numpy as np 
%matplotlib inline

2023-02-10 11:03:02.770167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 11:03:03.445417: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 11:03:03.445492: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-10 11:03:03.556462: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-10 11:03:06.765130: W tensorflow/stream_executor/platform/de

In [2]:
#Funzione per fare mirror orizzonale delle immagini e delle relative maschere
            #immagine, nome immagine, maschera, salvataggio immagine, salvataggio maschera
def mirror_hor(img,name_img,mask,dest_img, dest_mask):
    flip_img = ImageOps.mirror(img)
    flip_mask = ImageOps.mirror(mask)
    dest_img = os.path.join(dest_img,"trasl_"+name_img)
    dest_mask = os.path.join(dest_mask,"trasl_"+name_img)
    flip_img.save(dest_img)
    flip_mask.save(dest_mask)

In [3]:
#Funzione per cambiare il colore di un immagine (la maschera rimane uguale)
def color(img,name_img,mask,dest_img,dest_mask):
    img = img.convert("RGBA")
    factor = random.uniform(0.0,3.0) # fattore random per la modifica del colore
    new_img = ImageEnhance.Color(img).enhance(factor)
    
    #Save
    dest_img = os.path.join(dest_img,"col_"+name_img)
    dest_mask = os.path.join(dest_mask,"col_"+name_img)
    new_img.save(dest_img)
    mask.save(dest_mask)

In [4]:
#Funzione per aggiungere rumore gaussiano alle immagini (le maschere rimangono uguali)
#Aggiunta di pixel di vari colori sull'immagine
def gaussian_noise(image_path,name_img,mask,dest_img,dest_mask):
    path = os.path.join("..","original-dataset","images",name_img)
    img = cv2.imread(path) #unico che funziona
   
    # Generate Gaussian noise
    gauss = np.random.normal(0,0.6,img.size)
    gauss = gauss.reshape(img.shape[0],img.shape[1],img.shape[2]).astype('uint8')
    # Add the Gaussian noise to the image
    img_gauss = cv2.add(img,gauss)
    
    # Display the image
    cv2.imshow('a',img_gauss)
    cv2.waitKey(0)
    path2 = os.path.join("..","car-segmentation","images","gauss_"+name_img)
    cv2.imwrite(path2,img_gauss)
    dest_mask = os.path.join(dest_mask,"gauss_"+name_img)
    mask.save(dest_mask)

In [5]:
#Funzione per sfocare o definire un'immagine in modo random (maschere uguali)
def blur(img,name_img,mask,dest_img,dest_mask):
    img = img.convert("RGBA")
    
    factor = random.uniform(-2.0,4.0) # fattore random per la modifica del colore
    new_img = ImageEnhance.Sharpness(img).enhance(factor)
    #new_img.show()
    
    #Save
    dest_img = os.path.join(dest_img,"blur_"+name_img)
    dest_mask = os.path.join(dest_mask,"blur_"+name_img)
    new_img.save(dest_img)
    mask.save(dest_mask)
    

In [6]:
#Funzione per ruotare di 25° o -25° (random) ogni immagine e ogni maschera
def rotate(img,name_img,mask,dest_img,dest_mask):
    angle =  random.choice([-25, 25])
    new_img = img.rotate(angle)
    new_mask = mask.rotate(angle,fillcolor=0) #Colore di background della maschera
    #new_mask.show()
    
    #Save
    dest_img = os.path.join(dest_img,"rot_"+name_img)
    dest_mask = os.path.join(dest_mask,"rot_"+name_img)
    new_img.save(dest_img)
    new_mask.save(dest_mask)
    

In [7]:
#funzione per scalare le immagini
def scale_image(img, scale,name_img,mask,dest_img,dest_mask): 
  width = int(img.shape[1] * scale) 
  height = int(img.shape[0] * scale) 
  dim = (width, height) 
  
  # resize image 
  resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)  
  path = os.path.join("..","original-dataset","images",name_img)
  #read image file
  img = cv2.imread(path)
  # scale image
  new_image = scale_image(img, 0.5)
  new_mask= scale_image(mask, 0.5)
  # show image
  cv2.imshow("Resized image", resized)
  cv2.waitKey(0)
    #Save
  dest_img = os.path.join(dest_img,"scale_"+name_img)
  dest_mask = os.path.join(dest_mask,"scale_"+name_img)
  new_image.save(dest_img)
  new_mask.save(dest_mask)
  return resized




In [8]:
#Get Directory of car-segmentation images
parent = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) #parent directory
img_folder = os.path.join(parent, "original-dataset", "images")
masks_folder = os.path.join(parent, "original-dataset", "masks")
destination_img_folder = os.path.join(parent, "car-segmentation", "images") #cartella di destinazione su cui salvare le immagini
destination_mask_folder = os.path.join(parent, "car-segmentation", "masks") #cartella di destinazione su cui salvare le maschere 
all_images = os.listdir(img_folder)
all_masks = os.listdir(masks_folder)

for i in range(5): #len(all_images)
    print("Eseguo sull'immagine: ", i+1)
    #immagine
    file = all_images[i] #Nome dell'immagine (e della maschera)
    image_path = os.path.join(img_folder, file)
    img = Image.open(image_path)
    #maschera
    mask_path = os.path.join(masks_folder,file)
    mask = Image.open(mask_path)
    
    #Operazioni
    #1) Mirror orizzonale di ogni immagine
    mirror_hor(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #2) Rumore gaussiano
    gaussian_noise(image_path,file,mask,destination_img_folder,destination_mask_folder)
    
    #3) Colore
    color(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #4) Blur
    blur(img,file,mask,destination_img_folder,destination_mask_folder)
    
    #5) Rotate 
    rotate(img,file,mask,destination_img_folder,destination_mask_folder)
    
  

Eseguo sull'immagine:  1
Eseguo sull'immagine:  2
Eseguo sull'immagine:  3
Eseguo sull'immagine:  4
Eseguo sull'immagine:  5
